In [66]:
import influxdb_client, os, time
from influxdb import DataFrameClient
from datetime import datetime
import pandas as pd
from IPython.display import display, HTML,Markdown

In [73]:
#setting & init
token = "RIZZsgptkpmFaqmapGn55oDyEDKVC9kDS1FkY7iLNLUINCcnpTcVTRdDAFSWmVqjUBpSo5fWuDBCGRPHYUfO2A=="
org = "metaportalweb@gmail.com"
url = "https://us-east-1-1.aws.cloud2.influxdata.com"

ronin_treasury = "a99cacd1427f493a95b585a5c7989a08c86a616b"

client = influxdb_client.InfluxDBClient(url=url, token=token, org=org)

In [64]:
## helper function
def format_df(data):
    df = data.copy()
    df['_time'] = df['_time'].apply((lambda x: pd.to_datetime(x.value,unit="ms")))
    df = df.set_index('_time')
    df.index.name = 'date'
    df = df[['from','to','value']]
    return df
    
def printmd(string):
    display(Markdown(string))


In [127]:
query_api = client.query_api()
bucket = "WETH"

query = f"""from(bucket: "{bucket}")
 |> range(start: -10000000000m) 
 |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
"""


raw_weth = query_api.query_data_frame(query, org=org)
weth_df = format_df(raw_weth)

In [135]:
bucket = "AXS"
query = f"""from(bucket: "{bucket}")
 |> range(start: -10000000000m) 
 |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
"""


raw_axs = query_api.query_data_frame(query, org=org)
axs_df = format_df(raw_axs)

In [75]:
# find whales based on from tx

# WETH
weth_groupby_from = weth_df.groupby('from').sum().sort_values(by='value',ascending=False)
printmd("<div style='font-size:20px'>WETH Whales: </div>")
display(weth_groupby_from)
# AXS
axs_groupby_from = axs_df.groupby('from').sum().sort_values(by='value',ascending=False)
printmd("<div style='font-size:20px'>AXS Whales: </div>")
display(axs_groupby_from)


<div style='font-size:20px'>WETH Whales: </div>

,value
from,
d0fe56a20bc404cc743b9cad8d9245e2c658fe7e,1.655000e+03
6a0ac34a3726daff83a83e5e454a4a976e33ffac,4.588299e+02
70922ea1d3adb3fd05f9df07aa925827550ecd61,2.861162e+02
d53000053e91597add02e491536628c40c9072d8,2.340000e+02
0e8cb007d7fe291d275cd83c18b4a7eefa112d62,1.960000e+02
...,...
88b17ee35090439acf1bfb905617ccb0f26ac14d,9.996620e-13
bb007aa543514b1761d5e53058c1d23a6e696e14,1.111110e-13
b84438b49b6b672695eab37ec5a32d06ebdf3f2b,0.000000e+00


<div style='font-size:20px'>AXS Whales: </div>

,value
from,
b32e9a84ae0b55b8ab715e4ac793a61b277bafa3,5.009797e+05
030e37ddd7df1b43db172b23916d523f1599c6cb,3.455870e+05
01acad0e173d09236d2af7b013442de042827881,9.349129e+04
ecc7c35c522b4239a413f212c08d458561d004d2,8.038873e+04
5d8a9e08f14eb765b8b906cb7b7617da067d5ca1,8.036047e+04
...,...
279c3481874411d50b01635d0832a6aea4b6106e,1.615537e-08
21e7d9e5ff60dc3fc225bdb25fd808d558fab32d,1.284549e-09
5a3f3f55ee773584731674502810a4f6c3fa5081,0.000000e+00


In [76]:
# find whales based on to tx

# WETH
weth_groupby_to = weth_df.groupby('to').sum().sort_values(by='value',ascending=False)
printmd("<div style='font-size:20px'>WETH Whales: </div>")
display(weth_groupby_to)
# AXS
axs_groupby_to = axs_df.groupby('to').sum().sort_values(by='value',ascending=False)
printmd("<div style='font-size:20px'>AXS Whales: </div>")
display(axs_groupby_to)


<div style='font-size:20px'>WETH Whales: </div>

,value
to,
6a0ac34a3726daff83a83e5e454a4a976e33ffac,1.655000e+03
d0fe56a20bc404cc743b9cad8d9245e2c658fe7e,4.588289e+02
70922ea1d3adb3fd05f9df07aa925827550ecd61,2.470000e+02
553f59757e2e76cb96f457fcba38756c87b4d0b9,1.625915e+02
ad19f6ab7b487882b1ae244286c2cbebc76b64f0,1.380000e+02
...,...
bb297b294c9eae6718d151cc215407c6bcd1a53d,2.000000e-18
888045f89116c867c6fd0398c72a281980cc65f5,0.000000e+00
3afef28c419c4613cbe7d9bdd0a818d8aaa63a33,0.000000e+00


<div style='font-size:20px'>AXS Whales: </div>

,value
to,
b32e9a84ae0b55b8ab715e4ac793a61b277bafa3,7.605898e+05
3205b65a18d13360048bb90aad348ade9c06dbf3,3.455870e+05
609f5d055e9290457c8a06479bcaaa8902ddbdec,9.347306e+04
d31172d56e51078a1fdef1fea9210807ecc8bf1a,9.300099e+04
ecc7c35c522b4239a413f212c08d458561d004d2,8.038873e+04
...,...
f167b8d09556f5b8f0e5fc3468bd29b03ded93f9,5.635473e-08
05b7550b4ae4ce2185ca701ca62ba623ea221df0,5.159976e-08
926fda9042f447d4a9bf03ab828b1d67c6096682,0.000000e+00


In [98]:
#combine from and to find the whale based on associate tx

#combine 
weth_value_dict = weth_groupby_from.to_dict()
for (address,value) in weth_groupby_to.to_dict()['value'].items():
    cummulative_value = weth_value_dict['value'].get(address,0) + value
    weth_value_dict['value'][address] = cummulative_value

weth_combine_df = pd.DataFrame(weth_value_dict).sort_values(by='value',ascending=False)
weth_combine_df.index.name = 'address'

axs_value_dict = axs_groupby_from.to_dict()
for (address,value) in axs_groupby_to.to_dict()['value'].items():
    cummulative_value = axs_value_dict['value'].get(address,0) + value
    axs_value_dict['value'][address] = cummulative_value

axs_combine_df = pd.DataFrame(axs_value_dict).sort_values(by='value',ascending=False)
axs_combine_df.index.name = 'address'

printmd("<div style='font-size:20px'>WETH Whales: </div>")
display(weth_combine_df)

printmd("<div style='font-size:20px'>AXS Whales: </div>")
display(axs_combine_df)

<div style='font-size:20px'>WETH Whales: </div>

,value
address,
6a0ac34a3726daff83a83e5e454a4a976e33ffac,2113.829918
d0fe56a20bc404cc743b9cad8d9245e2c658fe7e,2113.828918
70922ea1d3adb3fd05f9df07aa925827550ecd61,533.116173
d53000053e91597add02e491536628c40c9072d8,346.616173
0e8cb007d7fe291d275cd83c18b4a7eefa112d62,196.000000
...,...
c8bae01eb82809128db75c8f9c72ed7b496f735e,0.000000
3afef28c419c4613cbe7d9bdd0a818d8aaa63a33,0.000000
888045f89116c867c6fd0398c72a281980cc65f5,0.000000


<div style='font-size:20px'>AXS Whales: </div>

,value
address,
b32e9a84ae0b55b8ab715e4ac793a61b277bafa3,1.261570e+06
030e37ddd7df1b43db172b23916d523f1599c6cb,3.455870e+05
3205b65a18d13360048bb90aad348ade9c06dbf3,3.455870e+05
ecc7c35c522b4239a413f212c08d458561d004d2,1.607775e+05
089ac289cada2b2b4156e24065b02aedcd19f2e9,1.600664e+05
...,...
5a3f3f55ee773584731674502810a4f6c3fa5081,0.000000e+00
37538e472a84520c3d7e3c713ae77fd5177a4bc8,0.000000e+00
926fda9042f447d4a9bf03ab828b1d67c6096682,0.000000e+00
